<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NM/Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install diffusers transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 806.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
# Установка необходимых библиотек
#!pip install diffusers transformers accelerate

from diffusers import DiffusionPipeline
import torch

# Выберите модель (например, Stable Diffusion)
model_id = "runwayml/stable-diffusion-v1-5"

# Загрузка модели
pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# Если у вас есть GPU, переместите

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# Если у вас есть GPU, переместите модель на него
if torch.cuda.is_available():
    pipe = pipe.to("cuda")

# Генерация изображения по текстовому описанию
prompt = "A beautiful sunset over the mountains"  # Ваше описание
image = pipe(prompt).images[0]

# Сохранение изображения
image.save("output_image.png")

  0%|          | 0/50 [00:00<?, ?it/s]

In [2]:
!pip install diffusers transformers accelerate torch torchvision pillow gradio scikit-image pytorch-fid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.27.1
    Uninstalling huggingface-hub-0.27.1:
      Successfully uninstalled huggingface-hub-0.27.1


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install pytorch-fid

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 664.8/664.8 MB 119.2 MB/s eta 0:00:01

In [3]:
# Установка необходимых библиотек
!pip install diffusers transformers accelerate torch torchvision pillow datasets tqdm scikit-image pytorch-fid

# Импорт библиотек
import os
import torch
from diffusers import UNet2DConditionModel, AutoencoderKL, DDPMScheduler, DiffusionPipeline
from transformers import CLIPTextModel, CLIPTokenizer
from datasets import load_dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from PIL import Image
import numpy as np
from pytorch_fid.fid_score import calculate_fid_given_paths
from skimage.metrics import structural_similarity as ssim

# Проверка доступности GPU
if not torch.cuda.is_available():
    raise RuntimeError("GPU недоступен. Убедитесь, что вы выбрали GPU в настройках Colab.")
else:
    print("CUDA доступен!")
    print("Текущее устройство:", torch.cuda.current_device())
    print("Название GPU:", torch.cuda.get_device_name(0))

# 1. Загрузка и подготовка датасета
def prepare_dataset(dataset_name="cifar10", split="train[:1%]"):
    # Загрузка датасета
    dataset = load_dataset(dataset_name, split=split)

    # Преобразования для изображений
    preprocess = transforms.Compose([
        transforms.Resize((512, 512)),  # Размер, требуемый Stable Diffusion
        transforms.ToTensor(),
    ])

    # Функция для обработки данных
    def transform_example(example):
        image = example["img"]
        image = preprocess(image.convert("RGB"))
        return {"image": image, "text": "A CIFAR-10 image"}  # Простое текстовое описание

    # Применение преобразований к датасету
    dataset = dataset.map(transform_example, remove_columns=dataset.column_names)
    return dataset

# 2. Fine-tuning U-Net
def fine_tune_model(dataset, epochs=2, batch_size=2, output_dir="./fine_tuned_unet"):
    # Создание DataLoader
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Загрузка компонентов модели
    model_id = "runwayml/stable-diffusion-v1-5"
    vae = AutoencoderKL.from_pretrained(model_id, subfolder="vae").to("cuda")
    tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder="text_encoder").to("cuda")
    unet = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet").to("cuda")

    # Шумовой планировщик
    noise_scheduler = DDPMScheduler.from_config(model_id)

    # Оптимизатор
    optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-5)

    # Цикл обучения
    for epoch in range(epochs):
        unet.train()
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}")
        for batch in progress_bar:
            images = batch["image"].to("cuda")
            captions = ["A CIFAR-10 image"] * len(images)

            # Токенизация текста
            inputs = tokenizer(captions, padding="max_length", truncation=True, max_length=77, return_tensors="pt")
            input_ids = inputs.input_ids.to("cuda")

            # Получение латентного представления изображений
            with torch.no_grad():
                latents = vae.encode(images).latent_dist.sample().detach()

            # Добавление шума
            noise = torch.randn(latents.shape).to("cuda")
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],)).to("cuda")
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # Предсказание U-Net
            encoder_hidden_states = text_encoder(input_ids)[0]
            noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

            # Вычисление loss
            loss = torch.nn.functional.mse_loss(noise_pred, noise)

            # Обратное распространение
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            progress_bar.set_postfix(loss=loss.item())

    # Сохранение обученной модели
    os.makedirs(output_dir, exist_ok=True)
    unet.save_pretrained(output_dir)
    print(f"U-Net успешно сохранен: {output_dir}")

# 3. Генерация изображений
def generate_images(pipe, prompts, output_dir="generated_images"):
    os.makedirs(output_dir, exist_ok=True)
    for i, prompt in enumerate(prompts):
        image = pipe(prompt).images[0]
        image.save(f"{output_dir}/image_{i}.png")

# 4. Вычисление метрик FID и SSIM
def calculate_metrics(real_images_path, generated_images_path):
    # Вычисление FID
    fid_value = calculate_fid_given_paths(
        paths=[real_images_path, generated_images_path],
        batch_size=2,
        device="cuda",
        dims=2048
    )
    print(f"FID: {fid_value}")

    # Вычисление SSIM для пары изображений
    def calculate_ssim(image1_path, image2_path):
        img1 = np.array(Image.open(image1_path).convert("L"))  # Конвертация в черно-белое
        img2 = np.array(Image.open(image2_path).convert("L"))
        return ssim(img1, img2)

    # Пример использования SSIM
    real_image_path = os.path.join(real_images_path, os.listdir(real_images_path)[0])
    generated_image_path = os.path.join(generated_images_path, os.listdir(generated_images_path)[0])
    ssim_value = calculate_ssim(real_image_path, generated_image_path)
    print(f"SSIM: {ssim_value}")

# Основной код
if __name__ == "__main__":
    # Подготовка датасета
    dataset = prepare_dataset()

    # Fine-tuning модели
    fine_tune_model(dataset)

    # Загрузка pipeline с обученной моделью
    pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
    pipe.unet = UNet2DConditionModel.from_pretrained("./fine_tuned_unet").to("cuda")
    pipe = pipe.to("cuda")

    # Генерация изображений
    prompts = ["A beautiful sunset", "A cat sitting on a chair", "A futuristic city"]
    generate_images(pipe, prompts)

    # Вычисление метрик
    real_images_path = "./real_images"  # Путь к папке с реальными изображениями
    generated_images_path = "./generated_images"  # Путь к папке с сгенерированными изображениями
    calculate_metrics(real_images_path, generated_images_path)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

ModuleNotFoundError: No module named 'datasets'

In [ ]:
# Установка необходимых библиотек
!pip install diffusers transformers accelerate torch torchvision pillow gradio pytorch-fid scikit-image datasets

# Импорт библиотек
import os
import torch
from diffusers import DiffusionPipeline, UNet2DConditionModel, AutoencoderKL, DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from datasets import load_dataset
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from pytorch_fid.fid_score import calculate_fid_given_paths
from skimage.metrics import structural_similarity as ssim
import numpy as np
from huggingface_hub import notebook_login

# 1. Аутентификация в Hugging Face
notebook_login()

# 2. Fine-tuning на собственных данных
def fine_tune_model(dataset_name="laion/laion-art", epochs=2, batch_size=2, output_dir="./fine_tuned_unet"):
    # Загрузка датасета
    dataset = load_dataset(dataset_name, split="train[:1%]")  # Используем 1% данных для быстрого тестирования

    # Преобразования для изображений
    preprocess = transforms.Compose([
        transforms.Resize((512, 512)),  # Размер, требуемый Stable Diffusion
        transforms.ToTensor(),
    ])

    # Функция для обработки данных
    def transform_example(example):
        image = example["image"]
        if isinstance(image, list):  # Если изображение представлено как список
            image = image[0]
        image = preprocess(image.convert("RGB"))
        return {"image": image, "text": example["caption"]}

    # Применение преобразований к датасету
    dataset = dataset.map(transform_example, remove_columns=dataset.column_names)

    # Создание DataLoader
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Загрузка компонентов модели
    model_id = "runwayml/stable-diffusion-v1-5"
    vae = AutoencoderKL.from_pretrained(model_id, subfolder="vae").to("cuda")
    tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(model_id, subfolder="text_encoder").to("cuda")
    unet = UNet2DConditionModel.from_pretrained(model_id, subfolder="unet").to("cuda")

    # Шумовой планировщик
    noise_scheduler = DDPMScheduler.from_config(model_id)

    # Оптимизатор
    optimizer = torch.optim.AdamW(unet.parameters(), lr=1e-5)

    # Цикл обучения
    for epoch in range(epochs):
        unet.train()
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch}")
        for batch in progress_bar:
            images = batch["image"].to("cuda")
            captions = batch["text"]

            # Токенизация текста
            inputs = tokenizer(captions, padding="max_length", truncation=True, max_length=77, return_tensors="pt")
            input_ids = inputs.input_ids.to("cuda")

            # Получение латентного представления изображений
            with torch.no_grad():
                latents = vae.encode(images).latent_dist.sample().detach()

            # Добавление шума
            noise = torch.randn(latents.shape).to("cuda")
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (latents.shape[0],)).to("cuda")
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # Предсказание U-Net
            encoder_hidden_states = text_encoder(input_ids)[0]
            noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

            # Вычисление loss
            loss = torch.nn.functional.mse_loss(noise_pred, noise)

            # Обратное распространение
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            progress_bar.set_postfix(loss=loss.item())

    # Сохранение обученной модели
    os.makedirs(output_dir, exist_ok=True)
    unet.save_pretrained(output_dir)
    print(f"U-Net успешно сохранен: {output_dir}")

# Пример использования
fine_tune_model()